# ***Step 1: Import libraries & Set up***

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 12 # N_mffc
hidden_size = 64
num_layers = 3
num_classes = 35
batch_size = 1
num_epochs = 25
learning_rate = 0.001

# ***Step 2: Use trained model (FP32)***
Reference: https://github.com/felixchenfy/Speech-Commands-Classification-by-LSTM-PyTorch

In [ ]:
# Recurrent neural network (many-to-one)
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        return x
model = LSTM(12, 35,64,3)
x = torch.randn(batch_size, sequence_length, input_size)
out = model(x)
print(out)



tensor([[[ 1.7637, -0.9800, -0.6555,  0.1932, -0.3409, -0.0665,  0.6504,
           0.7238,  1.0911,  0.6677, -0.8664, -0.2223],
         [-1.1098, -1.6612, -0.4038,  0.0420,  1.0753,  1.2533,  0.7596,
          -0.5717, -1.1137,  0.1827, -0.7476, -0.2680],
         [-0.6768, -0.6298,  0.2154,  0.4746,  1.9847, -1.0084,  0.9777,
           1.0625,  0.4091, -2.2603, -0.7325, -1.3566],
         [-0.1744, -0.9150, -0.5280, -1.3718,  0.1830, -0.4074,  1.2907,
           1.0388, -0.5333,  1.1896, -0.3074, -0.0305],
         [ 0.0448, -0.2148,  0.1973, -1.4013, -0.0332, -0.0995,  0.0424,
           1.6514, -0.3608,  1.6190,  1.7268, -0.1907],
         [-0.8449, -1.5990, -0.0937, -1.7475,  0.6126, -0.8535,  0.5097,
          -0.5163,  0.8922, -0.6304,  0.1695, -0.3058],
         [ 2.3129, -0.1987, -0.8219, -0.2416,  0.7688, -0.4140, -0.7907,
          -1.0445,  1.8149, -0.4881, -2.0722,  1.6015],
         [ 1.9237, -0.8907, -0.4955,  0.3595, -1.4627, -2.0675, -1.4387,
          -1.7042,  1.20

***--- Above LSTM model tensors representation for Affine per tensor quantization--***

In [ ]:
import os

In [ ]:
T = torch.tensor (out)
print(T)

tensor([[[ 1.7637, -0.9800, -0.6555,  0.1932, -0.3409, -0.0665,  0.6504,
           0.7238,  1.0911,  0.6677, -0.8664, -0.2223],
         [-1.1098, -1.6612, -0.4038,  0.0420,  1.0753,  1.2533,  0.7596,
          -0.5717, -1.1137,  0.1827, -0.7476, -0.2680],
         [-0.6768, -0.6298,  0.2154,  0.4746,  1.9847, -1.0084,  0.9777,
           1.0625,  0.4091, -2.2603, -0.7325, -1.3566],
         [-0.1744, -0.9150, -0.5280, -1.3718,  0.1830, -0.4074,  1.2907,
           1.0388, -0.5333,  1.1896, -0.3074, -0.0305],
         [ 0.0448, -0.2148,  0.1973, -1.4013, -0.0332, -0.0995,  0.0424,
           1.6514, -0.3608,  1.6190,  1.7268, -0.1907],
         [-0.8449, -1.5990, -0.0937, -1.7475,  0.6126, -0.8535,  0.5097,
          -0.5163,  0.8922, -0.6304,  0.1695, -0.3058],
         [ 2.3129, -0.1987, -0.8219, -0.2416,  0.7688, -0.4140, -0.7907,
          -1.0445,  1.8149, -0.4881, -2.0722,  1.6015],
         [ 1.9237, -0.8907, -0.4955,  0.3595, -1.4627, -2.0675, -1.4387,
          -1.7042,  1.20

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [ ]:
# Min-Max value of float_32 tensor (x) find out for scale (s) and zero point (z)

b = torch.max(T)
print(b)

tensor(3.0228)


In [ ]:
a = torch.min(T)
print(a)

tensor(-2.3842)


In [ ]:
# scale value

s = (b-a)/3

print(s)

tensor(1.8023)


In [ ]:
# zero point

z = torch.round(-a*3/(b-a))

print(z)

tensor(1.)


# ***Method-1:- Asymmetric (Affine) per tensor quantization scheme***

# ***Step-3: Apply Quantization by round and clipping function (Affine mapping)***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq = torch.clip(f, min=0, max=3) # Here min & max value we can change as per Tbit.
# But, I have checked for 2 bit
print (Tq)
torch.save(Tq,'qtz_tensor.pt')


tensor([[[2., 0., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.],
         [0., 0., 1., 1., 2., 2., 1., 1., 0., 1., 1., 1.],
         [1., 1., 1., 1., 2., 0., 2., 2., 1., -0., 1., 0.],
         [1., 0., 1., 0., 1., 1., 2., 2., 1., 2., 1., 1.],
         [1., 1., 1., 0., 1., 1., 1., 2., 1., 2., 2., 1.],
         [1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
         [2., 1., 1., 1., 1., 1., 1., 0., 2., 1., -0., 2.],
         [2., 1., 1., 1., 0., -0., 0., 0., 2., 2., 1., 0.],
         [1., 2., 1., 0., -0., 2., 0., 1., 1., 2., 1., 3.],
         [1., 1., 1., 1., 0., -0., 2., 2., 1., 2., 1., 2.],
         [0., 0., 1., 1., 0., 1., 1., 1., 1., 2., 1., 2.],
         [1., -0., 1., 1., -0., 2., 1., 0., 1., 2., 1., 0.],
         [1., 1., 1., 2., 0., 1., 0., 1., 1., -0., 2., 1.],
         [2., 0., 0., 1., 1., 1., 2., 1., 1., 1., -0., 1.],
         [1., 1., 1., 2., 0., 2., 2., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 2., 1., 2., 2., 1.],
         [2., 0., 1., 1., 1., 2., 1., 1., 2., 1

# ***Step-4: Apply Dequantization***

In [ ]:
Tdq = s*(Tq - z)
print(Tdq)

tensor([[[ 1.8023, -1.8023,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  1.8023,  0.0000,  0.0000,  0.0000],
         [-1.8023, -1.8023,  0.0000,  0.0000,  1.8023,  1.8023,  0.0000,
           0.0000, -1.8023,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  1.8023, -1.8023,  1.8023,
           1.8023,  0.0000, -1.8023,  0.0000, -1.8023],
         [ 0.0000, -1.8023,  0.0000, -1.8023,  0.0000,  0.0000,  1.8023,
           1.8023,  0.0000,  1.8023,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000, -1.8023,  0.0000,  0.0000,  0.0000,
           1.8023,  0.0000,  1.8023,  1.8023,  0.0000],
         [ 0.0000, -1.8023,  0.0000, -1.8023,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 1.8023,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          -1.8023,  1.8023,  0.0000, -1.8023,  1.8023],
         [ 1.8023,  0.0000,  0.0000,  0.0000, -1.8023, -1.8023, -1.8023,
          -1.8023,  1.80

# ***Step-5: MAE/MSE loss between T and Tdq***

***I. MAE loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 1.7637, -0.9800, -0.6555,  0.1932, -0.3409, -0.0665,  0.6504,
           0.7238,  1.0911,  0.6677, -0.8664, -0.2223],
         [-1.1098, -1.6612, -0.4038,  0.0420,  1.0753,  1.2533,  0.7596,
          -0.5717, -1.1137,  0.1827, -0.7476, -0.2680],
         [-0.6768, -0.6298,  0.2154,  0.4746,  1.9847, -1.0084,  0.9777,
           1.0625,  0.4091, -2.2603, -0.7325, -1.3566],
         [-0.1744, -0.9150, -0.5280, -1.3718,  0.1830, -0.4074,  1.2907,
           1.0388, -0.5333,  1.1896, -0.3074, -0.0305],
         [ 0.0448, -0.2148,  0.1973, -1.4013, -0.0332, -0.0995,  0.0424,
           1.6514, -0.3608,  1.6190,  1.7268, -0.1907],
         [-0.8449, -1.5990, -0.0937, -1.7475,  0.6126, -0.8535,  0.5097,
          -0.5163,  0.8922, -0.6304,  0.1695, -0.3058],
         [ 2.3129, -0.1987, -0.8219, -0.2416,  0.7688, -0.4140, -0.7907,
          -1.0445,  1.8149, -0.4881, -2.0722,  1.6015],
         [ 1.9237, -0.8907, -0.4955,  0.3595, -1.4627, -2.0675, -1.4387,
         

***II. MSE Loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 1.7637, -0.9800, -0.6555,  0.1932, -0.3409, -0.0665,  0.6504,
           0.7238,  1.0911,  0.6677, -0.8664, -0.2223],
         [-1.1098, -1.6612, -0.4038,  0.0420,  1.0753,  1.2533,  0.7596,
          -0.5717, -1.1137,  0.1827, -0.7476, -0.2680],
         [-0.6768, -0.6298,  0.2154,  0.4746,  1.9847, -1.0084,  0.9777,
           1.0625,  0.4091, -2.2603, -0.7325, -1.3566],
         [-0.1744, -0.9150, -0.5280, -1.3718,  0.1830, -0.4074,  1.2907,
           1.0388, -0.5333,  1.1896, -0.3074, -0.0305],
         [ 0.0448, -0.2148,  0.1973, -1.4013, -0.0332, -0.0995,  0.0424,
           1.6514, -0.3608,  1.6190,  1.7268, -0.1907],
         [-0.8449, -1.5990, -0.0937, -1.7475,  0.6126, -0.8535,  0.5097,
          -0.5163,  0.8922, -0.6304,  0.1695, -0.3058],
         [ 2.3129, -0.1987, -0.8219, -0.2416,  0.7688, -0.4140, -0.7907,
          -1.0445,  1.8149, -0.4881, -2.0722,  1.6015],
         [ 1.9237, -0.8907, -0.4955,  0.3595, -1.4627, -2.0675, -1.4387,
         

# ***Method-2:- Symmetric per tensor quantization scheme***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq1 = torch.clip(f, min=-2, max=2) # Here min & max value we can change as per Tbit.
# But, I have checked for 2 bit
print (Tq1)
torch.save(Tq1,'qtz_tensor.pt')


tensor([[[2., 0., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.],
         [0., 0., 1., 1., 2., 2., 1., 1., 0., 1., 1., 1.],
         [1., 1., 1., 1., 2., 0., 2., 2., 1., -0., 1., 0.],
         [1., 0., 1., 0., 1., 1., 2., 2., 1., 2., 1., 1.],
         [1., 1., 1., 0., 1., 1., 1., 2., 1., 2., 2., 1.],
         [1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
         [2., 1., 1., 1., 1., 1., 1., 0., 2., 1., -0., 2.],
         [2., 1., 1., 1., 0., -0., 0., 0., 2., 2., 1., 0.],
         [1., 2., 1., 0., -0., 2., 0., 1., 1., 2., 1., 3.],
         [1., 1., 1., 1., 0., -0., 2., 2., 1., 2., 1., 2.],
         [0., 0., 1., 1., 0., 1., 1., 1., 1., 2., 1., 2.],
         [1., -0., 1., 1., -0., 2., 1., 0., 1., 2., 1., 0.],
         [1., 1., 1., 2., 0., 1., 0., 1., 1., -0., 2., 1.],
         [2., 0., 0., 1., 1., 1., 2., 1., 1., 1., -0., 1.],
         [1., 1., 1., 2., 0., 2., 2., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 2., 1., 2., 2., 1.],
         [2., 0., 1., 1., 1., 2., 1., 1., 2., 1

In [ ]:
Tdq1 = s*(Tq1 - z)
print(Tdq1)

tensor([[[ 1.8023, -1.8023,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  1.8023,  0.0000,  0.0000,  0.0000],
         [-1.8023, -1.8023,  0.0000,  0.0000,  1.8023,  1.8023,  0.0000,
           0.0000, -1.8023,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  1.8023, -1.8023,  1.8023,
           1.8023,  0.0000, -1.8023,  0.0000, -1.8023],
         [ 0.0000, -1.8023,  0.0000, -1.8023,  0.0000,  0.0000,  1.8023,
           1.8023,  0.0000,  1.8023,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000, -1.8023,  0.0000,  0.0000,  0.0000,
           1.8023,  0.0000,  1.8023,  1.8023,  0.0000],
         [ 0.0000, -1.8023,  0.0000, -1.8023,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 1.8023,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          -1.8023,  1.8023,  0.0000, -1.8023,  1.8023],
         [ 1.8023,  0.0000,  0.0000,  0.0000, -1.8023, -1.8023, -1.8023,
          -1.8023,  1.80

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq1)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 1.7637, -0.9800, -0.6555,  0.1932, -0.3409, -0.0665,  0.6504,
           0.7238,  1.0911,  0.6677, -0.8664, -0.2223],
         [-1.1098, -1.6612, -0.4038,  0.0420,  1.0753,  1.2533,  0.7596,
          -0.5717, -1.1137,  0.1827, -0.7476, -0.2680],
         [-0.6768, -0.6298,  0.2154,  0.4746,  1.9847, -1.0084,  0.9777,
           1.0625,  0.4091, -2.2603, -0.7325, -1.3566],
         [-0.1744, -0.9150, -0.5280, -1.3718,  0.1830, -0.4074,  1.2907,
           1.0388, -0.5333,  1.1896, -0.3074, -0.0305],
         [ 0.0448, -0.2148,  0.1973, -1.4013, -0.0332, -0.0995,  0.0424,
           1.6514, -0.3608,  1.6190,  1.7268, -0.1907],
         [-0.8449, -1.5990, -0.0937, -1.7475,  0.6126, -0.8535,  0.5097,
          -0.5163,  0.8922, -0.6304,  0.1695, -0.3058],
         [ 2.3129, -0.1987, -0.8219, -0.2416,  0.7688, -0.4140, -0.7907,
          -1.0445,  1.8149, -0.4881, -2.0722,  1.6015],
         [ 1.9237, -0.8907, -0.4955,  0.3595, -1.4627, -2.0675, -1.4387,
         

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq1)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 1.7637, -0.9800, -0.6555,  0.1932, -0.3409, -0.0665,  0.6504,
           0.7238,  1.0911,  0.6677, -0.8664, -0.2223],
         [-1.1098, -1.6612, -0.4038,  0.0420,  1.0753,  1.2533,  0.7596,
          -0.5717, -1.1137,  0.1827, -0.7476, -0.2680],
         [-0.6768, -0.6298,  0.2154,  0.4746,  1.9847, -1.0084,  0.9777,
           1.0625,  0.4091, -2.2603, -0.7325, -1.3566],
         [-0.1744, -0.9150, -0.5280, -1.3718,  0.1830, -0.4074,  1.2907,
           1.0388, -0.5333,  1.1896, -0.3074, -0.0305],
         [ 0.0448, -0.2148,  0.1973, -1.4013, -0.0332, -0.0995,  0.0424,
           1.6514, -0.3608,  1.6190,  1.7268, -0.1907],
         [-0.8449, -1.5990, -0.0937, -1.7475,  0.6126, -0.8535,  0.5097,
          -0.5163,  0.8922, -0.6304,  0.1695, -0.3058],
         [ 2.3129, -0.1987, -0.8219, -0.2416,  0.7688, -0.4140, -0.7907,
          -1.0445,  1.8149, -0.4881, -2.0722,  1.6015],
         [ 1.9237, -0.8907, -0.4955,  0.3595, -1.4627, -2.0675, -1.4387,
         